In [1]:
import numpy as np
import itertools
import copy


In [2]:
n_vertices = 8

In [3]:
"""
Download presaved graphs for one less vertex
"""

n_vertices -= 1
file_str = 'graphs_' + str(n_vertices) + '.txt'

original_array = np.loadtxt(file_str)

#reshape array appropriately
num_entries = int(np.shape(original_array)[0] / n_vertices)
print('num_entries:',num_entries)
original_graphs = np.reshape(original_array,(num_entries,n_vertices,n_vertices))
original_graphs = original_graphs.tolist()

num_entries: 1044


In [4]:
original_graphs = [[[int(num) for num in sub] for sub in lst] for lst in original_graphs]
#print(original_graphs)

# PLAN

We have all the non-isomorphic graphs of size n-1
We append an extra vertex, and consider all the new adjacency matrices that may arise

Check if there's redundancies, delete if so


In [5]:
combos = [] #possible vertices to add an edge to new vertex
for k in range(n_vertices+1):
    which = list(itertools.combinations(range(n_vertices), k))
    for i in range(len(which)):
        combos.append(which[i])
print('Number of new adjacency matrices per original graph:', len(combos))
print('Total number of new adjacency matrices:',len(combos)*num_entries)

Number of new adjacency matrices per original graph: 128
Total number of new adjacency matrices: 133632


In [6]:
#initialize list of all distinct adjacency matrices
AllAdj = []

#go from combos to Adjacency matrix
for g in range(len(original_graphs)):
    for i in range(len(combos)):
        this_combo = combos[i]
        new_matrix = copy.deepcopy(original_graphs[g])
        
        new_matrix.append([0]*(n_vertices+1))
        for k in range(n_vertices):
            if k in this_combo:
                new_matrix[k].append(1)
                new_matrix[n_vertices][k] = 1
            else:
                new_matrix[k].append(0)
        
        AllAdj.append(new_matrix)

In [ ]:
"""
Delete Duplicates
"""
n_vertices += 1

def IsoGraphs(matrix1,matrix2,perm):
    """
    Suppose we permute the indices of matrix2 by perm
    Check if matrix1 = matrix2 permuted
    """
    n = len(matrix1)
    if len(matrix2) != n:
        return False
    
    #initially check if degrees line up
    for i in range(n):
        if sum(matrix1[i]) != sum(matrix2[perm[i]]):
            return False
    
    #then check if each entry lines up
    newmatrix2 = [[0]*n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            newmatrix2[i][j] = matrix2[perm[i]][perm[j]]
            if newmatrix2[i][j] != matrix1[i][j]:
                return False
    if matrix1 == newmatrix2:
        return True
    else:
        return False




"""
Compare 2 matrices to see if they're classically isomorphic.
Note: this is probably still a bad way to do it...

SHOULD BE IMPROVED
"""

#create list of vertices
vertices = []
for i in range(n_vertices):
    vertices.append(i)

# Get all permutations of [1, 2, 3,..., n_vertices] 
permutations = itertools.permutations(vertices) 
permlist = list(permutations)

matrix1 = 0
matrix2 = matrix1 + 1
while matrix1 < len(AllAdj):
    while matrix2 < len(AllAdj):
        #print('comparing matrix1:',matrix1,' and matrix2:',matrix2)
        matrix1iso2 = False
        p = 0
        while matrix1iso2 == False and p < len(permlist):
            #print('compare matrix1 with matrix2 edited by perm', permlist[p])
            matrix1iso2 = IsoGraphs(AllAdj[matrix1],AllAdj[matrix2],permlist[p])
            p += 1
        
        if matrix1iso2 == True:
            #print('they are isomorphic, delete matrix', matrix2)
            AllAdj.pop(matrix2) #delete matrix2 from list
        else:
            #print('they are not isomorphic')
            matrix2 += 1
    
    matrix1 += 1
    matrix2 = matrix1 + 1


print('\n\nFor ', n_vertices, 
      ' vertices, there are ', len(AllAdj), ' non-isomorphic graphs.')


In [ ]:
file_str = 'graphs_' + str(n_vertices) + '.txt'
a_file = open(file_str, "w")
for row in AllAdj:
    np.savetxt(a_file, row)

a_file.close()
